In [2]:
import pandas as pd
import plotly.express as px

# Results postprocessing

In [3]:
community = 'duluth' # TODO change community name
Total_dwelling_unit_count = 39762  # TODO supplied by the community

up00 = pd.read_csv(f'data_/community_building_samples_with_upgrade_cost_and_bill/{community}/up00__{community}.csv')
up01 = pd.read_csv(f'data_/community_building_samples_with_upgrade_cost_and_bill/{community}/up01__{community}.csv')
up02 = pd.read_csv(f'data_/community_building_samples_with_upgrade_cost_and_bill/{community}/up02__{community}.csv')
up03 = pd.read_csv(f'data_/community_building_samples_with_upgrade_cost_and_bill/{community}/up03__{community}.csv')
up04 = pd.read_csv(f'data_/community_building_samples_with_upgrade_cost_and_bill/{community}/up04__{community}.csv')
up05 = pd.read_csv(f'data_/community_building_samples_with_upgrade_cost_and_bill/{community}/up05__{community}.csv')
up06 = pd.read_csv(f'data_/community_building_samples_with_upgrade_cost_and_bill/{community}/up06__{community}.csv')
up07 = pd.read_csv(f'data_/community_building_samples_with_upgrade_cost_and_bill/{community}/up07__{community}.csv')
up08 = pd.read_csv(f'data_/community_building_samples_with_upgrade_cost_and_bill/{community}/up08__{community}.csv')
up09 = pd.read_csv(f'data_/community_building_samples_with_upgrade_cost_and_bill/{community}/up09__{community}.csv')
up10 = pd.read_csv(f'data_/community_building_samples_with_upgrade_cost_and_bill/{community}/up10__{community}.csv')


C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_18292\450388626.py:4: DtypeWarning: Columns (125) have mixed types. Specify dtype option on import or set low_memory=False.
  up00 = pd.read_csv(f'data_/community_building_samples_with_upgrade_cost_and_bill/{community}/up00__{community}.csv')
C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_18292\450388626.py:9: DtypeWarning: Columns (574,583,622,634,652,667) have mixed types. Specify dtype option on import or set low_memory=False.
  up05 = pd.read_csv(f'data_/community_building_samples_with_upgrade_cost_and_bill/{community}/up05__{community}.csv')
C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_18292\450388626.py:10: DtypeWarning: Columns (8,9,10,11,501,502,503,504,505,506,507,508,509,510,511,512,513,529) have mixed types. Specify dtype option on import or set low_memory=False.
  up06 = pd.read_csv(f'data_/community_building_samples_with_upgrade_cost_and_bill/{community}/up06__{community}.csv')
C:\Users\ylou2\AppData\Local\Temp\1\ipykernel_1829

## Housing characterization distribution

### 1. Vintage

In [11]:
# regroup input data from Jes
vintage_list = ['<1940', '1940s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s']
vintage_figure = ['Pre 1940', '1940-1979', 'Post 1980']
vintage_percentage = []
vintage_percentage_figure = []
for vintage in vintage_list:
    temp= up00.loc[up00['build_existing_model.vintage'] == vintage]
    temp_count =temp['sample_weight'].sum()/Total_dwelling_unit_count
    vintage_percentage.append(temp_count)
    
vintage_percentage_figure.append(vintage_percentage[0])
temp = vintage_percentage[1] + vintage_percentage[2] + vintage_percentage[3]+ vintage_percentage[4]
vintage_percentage_figure.append(temp)
temp = vintage_percentage[5] + vintage_percentage[6] + vintage_percentage[7]+ vintage_percentage[8]
vintage_percentage_figure.append(temp)

fig = px.bar(x=vintage_figure, y=vintage_percentage_figure, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=250, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  xaxis_title="Construction Year",
                  yaxis_title="Percentage",
                  yaxis_range=[0,0.4])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()

### 2. Building type

In [12]:
building_type_list = ['Single-Family Detached', 
                      'Single-Family Attached', 
                      'Multi-Family with 2 - 4 Units', 
                      'Multi-Family with 5+ Units',
                      'Mobile Home']
building_type_figure = ['Single-Family Detached', 'Single-Family Attached', 'Multi-Family 2-4 units', 'Multi-Family 5+ units', 'Mobile Home']
building_type_percentage = []
for building_type in building_type_list:
    temp = up00.loc[up00['build_existing_model.geometry_building_type_recs'] == building_type]
    temp_count =temp['sample_weight'].sum()/Total_dwelling_unit_count
    building_type_percentage.append(temp_count)
    
fig = px.bar(x=building_type_figure, y=building_type_percentage, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=320, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  xaxis_title="Building Type",
                  yaxis_title="Percentage",
                 yaxis_range=[0,0.8])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()

### 3. Heating fuel

In [14]:
heating_fuel_list = ['electricity', 'natural_gas', 'propane', 'fuel_oil']
heating_fuel_figure = ['Electricity', 'Natural Gas', 'Propane', 'Fuel Oil']
heating_fuel_consumption = []
for heating_fuel in heating_fuel_list:
    temp = up00[f'report_simulation_output.end_use_{heating_fuel}_heating_m_btu'].sum() 
    heating_fuel_consumption.append(temp)
    
total_energy = sum(heating_fuel_consumption)
heating_fuel_percentage = heating_fuel_consumption/total_energy

fig = px.bar(x=heating_fuel_figure, y=heating_fuel_percentage, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=290, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  xaxis_title="Heating Fuel",
                  yaxis_title="Percentage",
                 yaxis_range=[0,0.8])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()

### 4. Cooling type

In [16]:
cooling_type_list = list(up00['build_existing_model.hvac_cooling_type'].unique())
cooling_type_percentage = []
for cooling_type in cooling_type_list:
    temp = up00.loc[up00['build_existing_model.hvac_cooling_type'] == cooling_type]
    temp_count =temp['sample_weight'].sum()/Total_dwelling_unit_count
    cooling_type_percentage.append(temp_count)
    
fig = px.bar(x=cooling_type_list, y=cooling_type_percentage, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=290, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  xaxis_title="Cooling Type",
                  yaxis_title="Percentage",
                  yaxis_range=[0,0.6])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()

### 5. Envelope label

Envelope label criteria

In [18]:
# TODO, define the breakpoint
insulation_wall_r1 = 8
insulation_wall_r2 = 13
insulation_attic_r1 = 25
insulation_attic_r2 = 38
infiltration1 = 5
infiltration2 = 10

In [19]:
up00.loc[up00['build_existing_model.insulation_wall'] == 'Wood Stud, Uninsulated', 'build_existing_model.wall_assembly_r'] = 3.4
up00.loc[up00['build_existing_model.insulation_wall'] == 'Wood Stud, R-7', 'build_existing_model.wall_assembly_r'] = 8.7
up00.loc[up00['build_existing_model.insulation_wall'] == 'Wood Stud, R-11', 'build_existing_model.wall_assembly_r'] = 10.3
up00.loc[up00['build_existing_model.insulation_wall'] == 'Wood Stud, R-15', 'build_existing_model.wall_assembly_r'] = 12.1
up00.loc[up00['build_existing_model.insulation_wall'] == 'Wood Stud, R-19', 'build_existing_model.wall_assembly_r'] = 15.4
up00.loc[up00['build_existing_model.insulation_wall'] == 'CMU, 6-in Hollow, Uninsulated', 'build_existing_model.wall_assembly_r'] = 4
up00.loc[up00['build_existing_model.insulation_wall'] == 'CMU, 6-in Hollow, R-7', 'build_existing_model.wall_assembly_r'] = 9.4
up00.loc[up00['build_existing_model.insulation_wall'] == 'CMU, 6-in Hollow, R-11', 'build_existing_model.wall_assembly_r'] = 12.4
up00.loc[up00['build_existing_model.insulation_wall'] == 'CMU, 6-in Hollow, R-15', 'build_existing_model.wall_assembly_r'] = 15
up00.loc[up00['build_existing_model.insulation_wall'] == 'CMU, 6-in Hollow, R-19', 'build_existing_model.wall_assembly_r'] = 17.4
up00.loc[up00['build_existing_model.insulation_wall'] == 'Brick, 12-in, 3-wythe, Uninsulated', 'build_existing_model.wall_assembly_r'] = 4.9
up00.loc[up00['build_existing_model.insulation_wall'] == 'Brick, 12-in, 3-wythe, R-7', 'build_existing_model.wall_assembly_r'] = 10.3
up00.loc[up00['build_existing_model.insulation_wall'] == 'Brick, 12-in, 3-wythe, R-11', 'build_existing_model.wall_assembly_r'] = 13.3
up00.loc[up00['build_existing_model.insulation_wall'] == 'Brick, 12-in, 3-wythe, R-15', 'build_existing_model.wall_assembly_r'] = 15.9
up00.loc[up00['build_existing_model.insulation_wall'] == 'Brick, 12-in, 3-wythe, R-19', 'build_existing_model.wall_assembly_r'] = 18.3

up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Aluminum, Light', 'build_existing_model.wall_exterior_finish_r'] = 0.6
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Brick, Light', 'build_existing_model.wall_exterior_finish_r'] = 0.7
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Brick, Medium/Dark', 'build_existing_model.wall_exterior_finish_r'] = 0.7
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Fiber-Cement, Light', 'build_existing_model.wall_exterior_finish_r'] = 0.2
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Shingle, Composition, Medium', 'build_existing_model.wall_exterior_finish_r'] = 0.6
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Shingle, Asbestos, Medium', 'build_existing_model.wall_exterior_finish_r'] = 0.6
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Stucco, Light', 'build_existing_model.wall_exterior_finish_r'] = 0.2
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Stucco, Medium/Dark', 'build_existing_model.wall_exterior_finish_r'] = 0.2
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Vinyl, Light', 'build_existing_model.wall_exterior_finish_r'] = 0.6
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Wood, Medium/Dark', 'build_existing_model.wall_exterior_finish_r'] = 1.4
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'None', 'build_existing_model.wall_exterior_finish_r'] = 0

up00.loc[up00['build_existing_model.insulation_ceiling'] == 'None', 'build_existing_model.ceiling_assembly_r'] = 0
up00.loc[up00['build_existing_model.insulation_ceiling'] == 'Uninsulated', 'build_existing_model.ceiling_assembly_r'] = 2.1
up00.loc[up00['build_existing_model.insulation_ceiling'] == 'R-7', 'build_existing_model.ceiling_assembly_r'] = 8.7
up00.loc[up00['build_existing_model.insulation_ceiling'] == 'R-13', 'build_existing_model.ceiling_assembly_r'] = 14.6
up00.loc[up00['build_existing_model.insulation_ceiling'] == 'R-19', 'build_existing_model.ceiling_assembly_r'] = 20.6
up00.loc[up00['build_existing_model.insulation_ceiling'] == 'R-30', 'build_existing_model.ceiling_assembly_r'] = 31.6
up00.loc[up00['build_existing_model.insulation_ceiling'] == 'R-38', 'build_existing_model.ceiling_assembly_r'] = 39.6
up00.loc[up00['build_existing_model.insulation_ceiling'] == 'R-49', 'build_existing_model.ceiling_assembly_r'] = 50.6

up00.loc[up00['build_existing_model.insulation_roof'] == 'Unfinished, Uninsulated', 'build_existing_model.roof_assembly_r'] = 2.3
up00.loc[up00['build_existing_model.insulation_roof'] == 'Finished, Uninsulated', 'build_existing_model.roof_assembly_r'] = 3.7
up00.loc[up00['build_existing_model.insulation_roof'] == 'Finished, R-7', 'build_existing_model.roof_assembly_r'] = 10.2
up00.loc[up00['build_existing_model.insulation_roof'] == 'Finished, R-13', 'build_existing_model.roof_assembly_r'] = 14.3
up00.loc[up00['build_existing_model.insulation_roof'] == 'Finished, R-19', 'build_existing_model.roof_assembly_r'] = 21.2
up00.loc[up00['build_existing_model.insulation_roof'] == 'Finished, R-30', 'build_existing_model.roof_assembly_r'] = 29.7
up00.loc[up00['build_existing_model.insulation_roof'] == 'Finished, R-38', 'build_existing_model.roof_assembly_r'] = 36.5
up00.loc[up00['build_existing_model.insulation_roof'] == 'Finished, R-49', 'build_existing_model.roof_assembly_r'] = 47.0

up00['build_existing_model.insulation_wall_r'] = up00['build_existing_model.wall_assembly_r'].fillna(0) + up00['build_existing_model.wall_exterior_finish_r'].fillna(0)
up00['build_existing_model.insulation_attic_r'] = up00['build_existing_model.ceiling_assembly_r'].fillna(0) + up00['build_existing_model.roof_assembly_r'].fillna(0)

up00['build_existing_model.infiltration_new'] = up00['build_existing_model.infiltration'].str.replace(' ACH50', '')
up00['build_existing_model.infiltration_new'] = up00['build_existing_model.infiltration_new'].astype(float)

In [22]:
good = up00.loc[(up00['build_existing_model.insulation_wall_r'] > insulation_wall_r2) 
                & (up00['build_existing_model.insulation_attic_r'] > insulation_attic_r2) 
                & (up00['build_existing_model.infiltration_new'] > infiltration2)]
inadequate_and_good = up00.loc[(up00['build_existing_model.insulation_wall_r'] > insulation_wall_r1) 
                & (up00['build_existing_model.insulation_attic_r'] > insulation_attic_r1) 
                & (up00['build_existing_model.infiltration_new'] > infiltration1)]

total_count = len(up00)
good_count = len(good)/total_count
inadequate_count = (len(inadequate_and_good) - good_count)/total_count
poor_count = 1 - good_count - inadequate_count

fig = px.bar(x=['Good Envelope', 'Inadequate Envelope', 'Poor Envelope '], y=[good_count, inadequate_count, poor_count], text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=250, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  xaxis_title="Envelope Stauts",
                  yaxis_title="Percentage",
                  yaxis_range=[0,0.67])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()

get all the upgrades and their savings using process_annual_summary_for_dashboard.py 

In [29]:
total_type = 'report_utility_bills.bills_total_usd'
# report_simulation_output.energy_use_total_m_btu
# report_simulation_output.emissions_co_2_e_lrmer_95_decarb_by_2035_15_2025_start_total_lb
# report_utility_bills.bills_total_usd
total = []
total.append((up00[total_type]*up00['sample_weight']).sum())
total.append((up01[total_type]*up01['sample_weight']).sum())
total.append((up02[total_type]*up02['sample_weight']).sum())
total.append((up03[total_type]*up03['sample_weight']).sum())
total.append((up04[total_type]*up04['sample_weight']).sum())
total.append((up05[total_type]*up05['sample_weight']).sum())
total.append((up06[total_type]*up06['sample_weight']).sum())
total.append((up07[total_type]*up07['sample_weight']).sum())
total.append((up08[total_type]*up08['sample_weight']).sum())
total.append((up09[total_type]*up09['sample_weight']).sum())
total.append((up10[total_type]*up10['sample_weight']).sum())
print(f'Total savings for {total_type}')
for up in range(1, 11):
    saving = total[0] - total[up]
    saving_percentage = '{:,.1%}'.format((total[0] - total[up])/total[0])
    print(f'up{up}: {saving}')
    print(f'up{up}: {saving_percentage}')

Total savings for report_utility_bills.bills_total_usd
up1: 20756213.446008563
up1: 22.2%
up2: 22225572.842934698
up2: 23.8%
up3: -3276276.474993661
up3: -3.5%
up4: 16655304.988014653
up4: 17.8%
up5: 9930616.470872879
up5: 10.6%
up6: 2138576.101652488
up6: 2.3%
up7: 8108669.971870616
up7: 8.7%
up8: 25319167.02700928
up8: 27.1%
up9: 34993769.091286734
up9: 37.5%
up10: 93183973.55321954
up10: 99.9%


## Dwelling unit level by segment

Define segement

In [25]:
###### fuel
# 'electricity', 'natural_gas', 'fuel_oil', 'propane'
###### Vacancy Status
# 'Occupied', 'Vacant'

###### Building Type
# 'Mobile Home'
# 'Multi-Family with 2 - 4 Units'
# 'Multi-Family with 5+ Units'
# 'Single-Family Attached'
# 'Single-Family Detached'
###### Vintage
# '<1940', '1940s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s'
###### Tenure
# 'Renter', 'Owner'
###### AMI
# '0-80%','

######build_existing_model.geometry_wall_exterior_finish
#Brick

#####TODO, change the community needs follwing
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_vintage = ['<1940']
seg1_tenure = ['Renter']

seg2_building_type = ['Single-Family Detached']
seg2_vintage = ['<1940']
seg2_tenure = ['Owner']

seg3_building_type = ['Multi-Family with 2 - 4 Units']
seg3_vintage = ['1940s', '1950s', '1960s', '1970s']
seg3_tenure = ['Renter','Owner']

seg4_building_type = ['Multi-Family with 5+ Units']
seg4_vintage = ['1940s', '1950s', '1960s', '1970s']
seg4_tenure = ['Renter','Owner']

label each building sample

In [26]:
up00.loc[up00['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     up00['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     up00['build_existing_model.vintage'].isin(seg1_vintage) & 
                     up00['build_existing_model.tenure'].isin(seg1_tenure), 
         'build_existing_model.segment'] = 'segment1'
up00.loc[up00['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     up00['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     up00['build_existing_model.vintage'].isin(seg2_vintage) & 
                     up00['build_existing_model.tenure'].isin(seg2_tenure), 
         'build_existing_model.segment'] = 'segment2'
up00.loc[up00['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     up00['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     up00['build_existing_model.vintage'].isin(seg3_vintage) & 
                     up00['build_existing_model.tenure'].isin(seg3_tenure), 
         'build_existing_model.segment'] = 'segment3'
up00.loc[up00['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     up00['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     up00['build_existing_model.vintage'].isin(seg4_vintage) & 
                     up00['build_existing_model.tenure'].isin(seg4_tenure), 
         'build_existing_model.segment'] = 'segment4'

Get all the upgrades and their savings using process_annual_summary_for_dashboard.py 